In [1]:
import pandas as pd
# import numpy as np
import json
import os
import re

import sqlalchemy as sql
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Text, delete, insert, DateTime, Float, Boolean, select, distinct, inspect, Time
from datetime import datetime


In [29]:

# Set the DATABASE_URL environment variable
DATABASE_URL = os.environ.get("DATABASE_URL")
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

# Create the database engine
engine = sql.create_engine(DATABASE_URL)

metadata = MetaData()

# Query
def query(sql_query):
    data = pd.read_sql_query(sql_query, engine)
    data.index += 1
    return data

q = """
SELECT *
FROM aranet_lecture_data
"""

aranetExp = query(q)

q = """
SELECT *
FROM aranet4
"""

aranet4 = query(q)

# Ensure the Date and Time columns are of type string
aranet4['Date'] = aranet4['Date'].astype(str)
aranet4['Time'] = aranet4['Time'].astype(str)

# Combine Date and Time into a single datetime column
aranet4['Datetime'] = pd.to_datetime(aranet4['Date'] + ' ' + aranet4['Time'])

Here's a plan for the function:

1. Define a function `calculate_rolling` that takes four parameters: `dataframe`, `column_names`, `window_sizes`, and `operations`.
2. Inside the function, iterate over each `column_name` in `column_names`.
3. For each `column_name`, iterate over each `window_size` in `window_sizes`.
4. For each `window_size`, iterate over each `operation` in `operations`.
5. For each `operation`, calculate the rolling operation of the `column_name` with the `window_size` using the `rolling` method of the dataframe.
6. Use an if-elif-else statement to decide which operation to apply based on the `operation` parameter.
7. Store the result in a new column in the dataframe with a name that combines the `column_name`, `window_size`, and `operation`.
8. After all iterations, return the modified dataframe.

Here's the Python code for the function:



In [19]:
def calculate_rolling(dataframe, column_names, window_sizes, operations):
    for column_name in column_names:
        for window_size in window_sizes:
            for operation in operations:
                try:
                    dataframe[f'{column_name}_rolling_{operation}_{window_size}'] = getattr(dataframe[column_name].rolling(window=window_size, min_periods=1), operation)()
                except Exception as e:
                    print(f"Operation '{operation}' not supported: {str(e)}")
    return dataframe



You can use this function with your `main_features` list, a list of window sizes, and a list of operations. For example:



In [21]:
main_features = ['Carbon dioxide(ppm)', 'Temperature(°F)', 'Relative humidity(%)', 'Atmospheric pressure(hPa)']
window_sizes = [5, 15, 30, 60]
operations = ['mean', 'max', 'sum', 'std', 'count']
aranet4_mod = aranet4.copy()
aranet4_mod = calculate_rolling(aranet4_mod, main_features, window_sizes, operations)

aranet4_mod.info()

Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object has no attribute 'diff'
Operation 'diff' not supported: 'Rolling' object ha

The `rolling` function in pandas provides a variety of operations that can be performed on a rolling window of data. Here are some of them:

1. `mean()`: Compute the rolling mean of the data.
2. `sum()`: Compute the rolling sum of the data.
3. `min()`: Compute the rolling minimum of the data.
4. `max()`: Compute the rolling maximum of the data.
5. `std()`: Compute the rolling standard deviation of the data.
6. `var()`: Compute the rolling variance of the data.
7. `median()`: Compute the rolling median of the data.
8. `quantile()`: Compute the rolling quantile of the data.
9. `apply()`: Apply a function of your choice to the rolling window of data.
10. `corr()`: Compute the rolling correlation between two series.
11. `cov()`: Compute the rolling covariance between two series.
12. `skew()`: Compute the rolling skewness of the data.
13. `kurt()`: Compute the rolling kurtosis of the data.
14. `count()`: Count the number of non-NA values in the rolling window of data.

You can modify the `calculate_rolling` function to include any of these operations.

We can likewise use the `diff()` method to track the minute-to-minute differences in between records. This is effectively measureing the minute-to-minute rate of change. Because our smallest measurement of time is units of minutes, we use this as our smallest measure for our difference calculus. The distribution of the minute-to-minute rate of change values is approximately normal and centered about mean zero.

- `diff(1)` is the `one` minute difference (or the difference of the current record with the `1st` previous record).
- `diff(2)` is the `two` minute difference (or the difference of the current record with `2nd` previous record).
- `diff(n)` is the `n` minute difference (or the difference of the current record with `nth` previous record).


I question whether anything  beyond `diff(5)` is relevant. What is interesting is the pattern formed by tracking the consecutive magnitude and sign of the difference. Much of this can be captured with `diff(1)` alone though and watching the rolling statistics as well as the sign change patterns of `diff(1)`

We can additionally take the difference of the difference which could lead to patterns of continuous accelerated change in the monitored features.

Given the structure of our air monitor and the fact it is stable and note moving the vast majority of the time, we can also consider this be the a measure of a volume or cross-section of the air-flow through (into) the sensor chamber. The difference being a measure of the finite difference velocity at which `CO2 (ppm)` flows into, out of, or not at all from minute to minute. The second difference is the finite difference acceleration.

In [50]:
import pandas as pd

# Create a time series
df = pd.DataFrame({'Position': [1, 2, 4, 7, 11, 16]})

# Calculate the first difference (velocity)
df['Velocity'] = df['Position'].diff()

# Calculate the second difference (acceleration)
df['Acceleration'] = df['Velocity'].diff()

print(df)

   Position  Velocity  Acceleration
0         1       NaN           NaN
1         2       1.0           NaN
2         4       2.0           1.0
3         7       3.0           1.0
4        11       4.0           1.0
5        16       5.0           1.0



We want to make a function similar to `calculate_rolling()` for `calculate_diff()`.

In [31]:
def calculate_diff(dataframe, column_names, window_sizes):
    for column_name in column_names:
        for window_size in window_sizes:
            dataframe[f'{column_name}_diff_{window_size}'] = dataframe[column_name].diff(periods=window_size)
    return dataframe

In [40]:

main_features = ['Carbon dioxide(ppm)']
                #  , 'Temperature(°F)', 'Relative humidity(%)', 'Atmospheric pressure(hPa)']
window_sizes = [1]
                # , 2, 3, 4, 5]

# operations = ['mean', 'max', 'sum', 'std', 'count']

# Calculate the first difference (velocity)
aranet4_diff = aranet4.copy()
aranet4_diff = calculate_diff(aranet4_diff, main_features, window_sizes)

# Calculate the second difference (acceleration)
main_features= ['Carbon dioxide(ppm)_diff_1']
aranet4_diff = calculate_diff(aranet4_diff, main_features, window_sizes)
aranet4_diff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83432 entries, 1 to 83432
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Carbon dioxide(ppm)                83432 non-null  int64         
 1   Temperature(°F)                    83432 non-null  float64       
 2   Relative humidity(%)               83379 non-null  float64       
 3   Atmospheric pressure(hPa)          80910 non-null  float64       
 4   Date                               83432 non-null  object        
 5   Time                               83432 non-null  object        
 6   Datetime                           83432 non-null  datetime64[ns]
 7   Carbon dioxide(ppm)_diff_1         83431 non-null  float64       
 8   Carbon dioxide(ppm)_diff_1_diff_1  83430 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(2)
memory usage: 5.7+ MB


In [45]:
main_features = ['Carbon dioxide(ppm)', 'Carbon dioxide(ppm)_diff_1', 'Carbon dioxide(ppm)_diff_1_diff_1']
                #  , 'Temperature(°F)', 'Relative humidity(%)', 'Atmospheric pressure(hPa)']
window_sizes = [5]

operations = ['mean', 'max', 'sum', 'std']

aranet4_diff_rolling = aranet4_diff.copy()

aranet4_diff_rolling = calculate_rolling(aranet4_diff_rolling, main_features, window_sizes, operations)


In [46]:
aranet4_diff_rolling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83432 entries, 1 to 83432
Data columns (total 21 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   Carbon dioxide(ppm)                               83432 non-null  int64         
 1   Temperature(°F)                                   83432 non-null  float64       
 2   Relative humidity(%)                              83379 non-null  float64       
 3   Atmospheric pressure(hPa)                         80910 non-null  float64       
 4   Date                                              83432 non-null  object        
 5   Time                                              83432 non-null  object        
 6   Datetime                                          83432 non-null  datetime64[ns]
 7   Carbon dioxide(ppm)_diff_1                        83431 non-null  float64       
 8   Carbon dioxide(ppm)_diff_1

In [47]:
aranet4_diff_rolling.head(20)

,Carbon dioxide(ppm),Temperature(°F),Relative humidity(%),Atmospheric pressure(hPa),Date,Time,Datetime,Carbon dioxide(ppm)_diff_1,Carbon dioxide(ppm)_diff_1_diff_1,Carbon dioxide(ppm)_rolling_mean_5,...,Carbon dioxide(ppm)_rolling_sum_5,Carbon dioxide(ppm)_rolling_std_5,Carbon dioxide(ppm)_diff_1_rolling_mean_5,Carbon dioxide(ppm)_diff_1_rolling_max_5,Carbon dioxide(ppm)_diff_1_rolling_sum_5,Carbon dioxide(ppm)_diff_1_rolling_std_5,Carbon dioxide(ppm)_diff_1_diff_1_rolling_mean_5,Carbon dioxide(ppm)_diff_1_diff_1_rolling_max_5,Carbon dioxide(ppm)_diff_1_diff_1_rolling_sum_5,Carbon dioxide(ppm)_diff_1_diff_1_rolling_std_5
1,1014,77.8,47.0,1007.3,01/12/2024,16:08:15,2024-01-12 16:08:15,NaN,NaN,1014.000000,...,1014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,884,78.5,46.0,1007.3,01/12/2024,16:09:15,2024-01-12 16:09:15,-130.0,NaN,949.000000,...,1898.0,91.923882,-130.0,-130.0,-130.0,NaN,NaN,NaN,NaN,NaN
3,914,79.7,43.0,1007.4,01/12/2024,16:10:15,2024-01-12 16:10:15,30.0,160.0,937.333333,...,2812.0,68.068593,-50.0,30.0,-100.0,113.137085,160.0,160.0,160.0,NaN
4,906,79.7,42.0,1007.2,01/12/2024,16:11:15,2024-01-12 16:11:15,-8.0,-38.0,929.500000,...,3718.0,57.743687,-36.0,30.0,-108.0,83.594258,61.0,160.0,122.0,140.007143
5,896,79.3,41.0,1007.3,01/12/2024,16:12:15,2024-01-12 16:12:15,-10.0,-2.0,922.800000,...,4614.0,52.203448,-29.5,30.0,-118.0,69.481412,40.0,160.0,120.0,105.470375
6,862,78.6,41.0,1007.1,01/12/2024,16:13:15,2024-01-12 16:13:15,-34.0,-24.0,892.400000,...,4462.0,20.366639,-30.4,30.0,-152.0,60.206312,24.0,160.0,96.0,91.869473
7,917,78.1,41.0,1007.2,01/12/2024,16:14:15,2024-01-12 16:14:15,55.0,89.0,899.000000,...,4495.0,22.226111,6.6,55.0,33.0,35.451375,37.0,160.0,185.0,84.705372
8,897,77.5,41.0,1007.1,01/12/2024,16:15:15,2024-01-12 16:15:15,-20.0,-75.0,895.600000,...,4478.0,20.598544,-3.4,55.0,-17.0,34.231564,-10.0,89.0,-50.0,61.379964
9,883,76.9,41.0,1007.2,01/12/2024,16:16:15,2024-01-12 16:16:15,-14.0,6.0,891.000000,...,4455.0,20.260800,-4.6,55.0,-23.0,34.536937,-1.2,89.0,-6.0,59.486973
10,893,76.5,42.0,1007.1,01/12/2024,16:17:15,2024-01-12 16:17:15,10.0,24.0,890.400000,...,4452.0,20.119642,-0.6,55.0,-3.0,34.911316,4.0,89.0,20.0,60.526854
